In [ ]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
import re
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.simple import CharTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn

In [ ]:
# loading model data created in data exploration

model_data = pd.read_csv("data/nfl_comments_top_10_sampled.csv", na_values={'author': [], "body": []}, keep_default_na= False)

train, test = train_test_split(model_data, stratify=model_data['author'], random_state = 187, test_size=0.1)

train_x = train['body']
train_y = train['author']
test_x = test['body']
test_y = test['author']

In [ ]:
# tokenize data in comment body using Penn Treebank tokenizer

tokenizer = TreebankWordTokenizer()

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [ ]:
# word canonicalization

def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    #word = word.lower()
    #replace urls with token
    word = re.sub(r"//\S+", "<url>", word)
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return constants.UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

train_x_tokens_canonicalize = []
test_x_tokens_canonicalize = []

for x in train_x_tokens:
    train_x_tokens_canonicalize.append(canonicalize_words(x))
    
for x in test_x_tokens:
    test_x_tokens_canonicalize.append(canonicalize_words(x))

In [ ]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens_canonicalize for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=None)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens_canonicalize:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens_canonicalize:
    test_x_ids.append(vocab.words_to_ids(x))

In [ ]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size

for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix

train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes BoW for simple baseline model

nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: ",acc)

In [ ]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model

lb = preprocessing.LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
lb.fit(np.asarray(train_y.tolist()).flatten())
    
train_y_neural = lb.transform(np.asarray(train_y.tolist()))
test_y_neural = lb.transform(np.asarray(test_y.tolist()))

In [ ]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 50, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on test set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_test = np.array(test_x_neural)
y_test = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_test", np.shape(x_test))
print("y_test", np.shape(y_test))

print("Vocabulary Size: {:d}".format(vocab.size))

# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=10,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Test summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "test")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_test, y_test, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

In [ ]:
# creating character level inputs for character based CNN model

# tokenize data in comment body using nltk char tokenizer

tokenizer_char = CharTokenizer()

train_x_tokens_char = []
test_x_tokens_char = []

for x in train_x:
    train_x_tokens_char.append(tokenizer_char.tokenize(x))
    
for x in test_x:
    test_x_tokens_char.append(tokenizer_char.tokenize(x))
    
# define character-vocabulary using w266 common vocab

train_text_all_char = [item for sublist in train_x_tokens_char for item in sublist]

vocab_char = vocabulary.Vocabulary(train_text_all_char, size=None)
print("Vocabulary size: {:,}".format(vocab_char.size))
print("Vocabulary dict: ", vocab_char.word_to_id)

train_x_ids_char = []
test_x_ids_char = []

for x in train_x_tokens_char:
    train_x_ids_char.append(vocab_char.words_to_ids(x))
    
for x in test_x_tokens_char:
    test_x_ids_char.append(vocab_char.words_to_ids(x))
    
# pad all id vectors to max length for character Neural model

max_comment_length_char = max([len(x) for x in train_x_ids_char])

train_x_neural_char = train_x_ids_char
test_x_neural_char = test_x_ids_char

for x in train_x_neural_char:
    if len(x) < max_comment_length_char:
        for n in range(max_comment_length_char - len(x)):
            x.append(1)
    elif len(x) > max_comment_length_char:
        x = x[:max_comment_length_char]
        
for x in test_x_neural_char:
    if len(x) < max_comment_length_char:
        for n in range(max_comment_length_char - len(x)):
            x.append(1)
    elif len(x) > max_comment_length_char:
        x = x[:max_comment_length_char]

In [ ]:
# creating and training character CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", vocab_char.size, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 50, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on test set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train_char = np.array(train_x_neural_char)
y_train_char = np.array(train_y_neural)
x_test_char = np.array(test_x_neural_char)
y_test_char = np.array(test_y_neural)

print("x_train", np.shape(x_train_char))
print("y_train", np.shape(y_train_char))
print("x_test", np.shape(x_test_char))
print("y_test", np.shape(y_test_char))

print("Vocabulary Size: {:d}".format(vocab_char.size))

# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train_char.shape[1],
            num_classes=10,
            vocab_size=vocab_char.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Test summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "test")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train_char, y_train_char)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_test_char, y_test_char, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))